<a href="https://colab.research.google.com/github/michnemi/COVPH-rates-and-policies/blob/main/covid_and_policies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DOH data daily cases

In [1]:
import pandas as pd
from datetime import datetime


In [2]:
covid_cases = pd.read_csv("/content/drive/MyDrive/MA in Economics/Econ 232 Econometrics/term_paper/DOH COVID Data Drop_ 20210516 - 04 Case Information 2020.csv")

In [3]:
covid_cases.tail()

,CaseCode,Age,AgeGroup,Sex,DateSpecimen,DateResultRelease,DateRepConf,DateDied,DateRecover,RemovalType,Admitted,RegionRes,ProvRes,CityMunRes,CityMuniPSGC,BarangayRes,BarangayPSGC,HealthStatus,Quarantined,DateOnset,Pregnanttab,ValidationStatus
473648,C883143,45.0,45 to 49,FEMALE,2020-09-28,2020-09-30,2020-10-02,NaN,2020-10-09,RECOVERED,YES,Region IV-B: MIMAROPA,ORIENTAL MINDORO,CITY OF CALAPAN (CAPITAL),PH175205000,SAN VICENTE SOUTH (POB.),PH175205053,RECOVERED,YES,NaN,NO,NaN
473649,C518857,22.0,20 to 24,FEMALE,2020-09-17,2020-09-20,2020-09-23,NaN,2020-10-02,RECOVERED,YES,Region IV-B: MIMAROPA,ORIENTAL MINDORO,CITY OF CALAPAN (CAPITAL),PH175205000,SAPUL,PH175205058,RECOVERED,YES,NaN,NO,Date Admitted is after Died/Recovered Date
473650,C119705,59.0,55 to 59,MALE,2020-10-10,2020-10-13,2020-10-15,NaN,NaN,RECOVERED,YES,Region IV-B: MIMAROPA,PALAWAN,CUYO,PH175310000,CABIGSING (POB.),PH175310003,RECOVERED,YES,2020-10-07,NaN,"Removal Type is ""Recovered"", but no Recovered ..."
473651,C785812,25.0,25 to 29,FEMALE,2020-09-10,2020-09-11,2020-09-13,NaN,2020-10-01,RECOVERED,YES,Region IV-B: MIMAROPA,PALAWAN,PUERTO PRINCESA CITY (CAPITAL),PH175316000,SAN JOSE,PH175316046,RECOVERED,YES,NaN,NO,NaN
473652,C360508,23.0,20 to 24,FEMALE,2020-08-27,2020-09-02,2020-09-04,NaN,NaN,RECOVERED,YES,Region IV-B: MIMAROPA,ROMBLON,SAN ANDRES,PH175912000,MATUTUNA,PH175912008,RECOVERED,YES,NaN,NO,"Health Status is ""Recovered"", but no Date Reco..."


In [4]:
covid_cases[['CaseCode', 'DateSpecimen', 'RemovalType']].dropna().shape

(349730, 3)

In [5]:
# extract only the needed info
covid_cases_extracted = covid_cases[['CaseCode', 'DateSpecimen', 'RemovalType', 'DateResultRelease', 'DateRepConf', 'RegionRes']]
covid_cases_extracted = covid_cases_extracted.fillna('')
covid_cases_extracted['DateSpecimen'] = covid_cases_extracted['DateSpecimen'].apply(lambda x: \
                                                datetime.strptime(str(x), '%Y-%m-%d') if x!='' else '')
covid_cases_extracted['DateResultRelease'] = covid_cases_extracted['DateResultRelease'].apply(lambda x: \
                                                datetime.strptime(str(x), '%Y-%m-%d') if x!='' else '')
covid_cases_extracted['DateRepConf'] = covid_cases_extracted['DateRepConf'].apply(lambda x: \
                                                datetime.strptime(str(x), '%Y-%m-%d') if x!='' else '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  """


In [6]:
covid_cases_extracted['case_date'] = covid_cases_extracted.apply(lambda x: min([i for i in [x['DateSpecimen'], x['DateResultRelease'], 
                                                                x['DateRepConf']] if pd.isna(i)==0]), axis=1)

In [7]:
region = covid_cases_extracted.groupby('RegionRes').agg({'CaseCode': len})

# Covid Policies

In [8]:
policies = pd.read_csv("/content/drive/MyDrive/MA in Economics/Econ 232 Econometrics/term_paper/policy_data - Sheet2.csv")

## Convert dates

In [9]:
policies['date_parsed'] = policies['date'].apply(lambda x: datetime.strptime(x, '%B %d, %Y'))

In [10]:
policies.date_parsed

0    2020-01-23
1    2020-01-28
2    2020-01-31
3    2020-02-02
4    2020-02-10
        ...    
78   2020-12-15
79   2020-12-23
80   2020-12-26
81   2020-12-28
82   2020-12-29
Name: date_parsed, Length: 83, dtype: datetime64[ns]

In [11]:
# create timeline for policies
pt_ = []

for i in policies.iterrows():
  row = i[1]

  srs = pd.Series(pd.date_range(row['date_parsed'], '2020-12-31', freq='D')).rename('date')
  srs = pd.DataFrame(srs)
  srs['category'] = row['category']
  
  pt_.append(srs)

policies_timeline = pd.concat(pt_)
policies_timeline = policies_timeline.groupby('date').agg({'category':list})
# policies_timeline = policies_timeline.groupby('date').agg({'category':lambda x: list(set(x))})


In [12]:
categories = policies.category.unique().tolist()

In [13]:
# count number of policies implemented as of date
for c_ in categories:
  policies_timeline[c_] = policies_timeline['category'].apply(lambda x: x.count(c_))

In [14]:
policies_timeline.tail()

,category,travel ban,travel ban lift,ease,postpone,emergency declaration,aid,protocol,testing,reopen,reporting change
date,,,,,,,,,,,
2020-12-27,"[travel ban, travel ban, travel ban, travel ba...",11,1,14,17,3,3,12,7,11,2
2020-12-28,"[travel ban, travel ban, travel ban, travel ba...",11,1,14,18,3,3,12,7,11,2
2020-12-29,"[travel ban, travel ban, travel ban, travel ba...",12,1,14,18,3,3,12,7,11,2
2020-12-30,"[travel ban, travel ban, travel ban, travel ba...",12,1,14,18,3,3,12,7,11,2
2020-12-31,"[travel ban, travel ban, travel ban, travel ba...",12,1,14,18,3,3,12,7,11,2


# Merge the two datasets

In [15]:
policies_timeline = policies_timeline.reset_index()
merged = covid_cases_extracted.merge(policies_timeline, how='outer', left_on='case_date', right_on='date')

In [16]:
merged.head(2)

,CaseCode,DateSpecimen,RemovalType,DateResultRelease,DateRepConf,RegionRes,case_date,date,category,travel ban,travel ban lift,ease,postpone,emergency declaration,aid,protocol,testing,reopen,reporting change
0,C908957,2020-06-25,RECOVERED,2020-06-25,2020-06-27,NCR,2020-06-25,2020-06-25,"[travel ban, travel ban, travel ban, travel ba...",9,1,1,11,2,3,1,6,5,1
1,C483823,2020-06-25,RECOVERED,NaT,2020-07-08,Region VIII: Eastern Visayas,2020-06-25,2020-06-25,"[travel ban, travel ban, travel ban, travel ba...",9,1,1,11,2,3,1,6,5,1


In [17]:
import numpy as np

In [18]:
agg_dict = {c_:max for c_ in categories}
agg_dict['CaseCode'] = list
agg_dict['RemovalType'] = list
covid_cases_daily = merged.groupby('date').agg(agg_dict)
covid_cases_daily['CaseCnt'] = covid_cases_daily['CaseCode'].apply(lambda x: len([1 for _ in x if not pd.isna(_)]))
covid_cases_daily['RemovalCnt'] = covid_cases_daily['RemovalType'].apply(lambda x: len([1 for _ in x if not pd.isna(_)]))

In [19]:
covid_cases_daily.tail(3)

,travel ban,travel ban lift,ease,postpone,emergency declaration,aid,protocol,testing,reopen,reporting change,CaseCode,RemovalType,CaseCnt,RemovalCnt
date,,,,,,,,,,,,,,
2020-12-29,12,1,14,18,3,3,12,7,11,2,"[C443925, C601544, C158602, C722338, C792657, ...","[RECOVERED, RECOVERED, RECOVERED, RECOVERED, R...",382,382
2020-12-30,12,1,14,18,3,3,12,7,11,2,"[C676358, C795693, C415494, C878696, C437731, ...","[DIED, RECOVERED, RECOVERED, RECOVERED, RECOVE...",290,290
2020-12-31,12,1,14,18,3,3,12,7,11,2,"[C341025, C352062, C359056, C623135, C529531, ...","[RECOVERED, RECOVERED, RECOVERED, RECOVERED, R...",387,387


In [20]:
covid_cases_daily = covid_cases_daily.reset_index()

In [21]:
covid_cases_daily['active_cases'] = covid_cases_daily.apply(lambda x: 
                                      covid_cases_daily[covid_cases_daily['date']<=x['date']]['CaseCnt'].sum()-\
                                      covid_cases_daily[covid_cases_daily['date']<=x['date']]['RemovalCnt'].sum(), 
                                      axis=1)

In [22]:
covid_cases_daily.tail(3)

,date,travel ban,travel ban lift,ease,postpone,emergency declaration,aid,protocol,testing,reopen,reporting change,CaseCode,RemovalType,CaseCnt,RemovalCnt,active_cases
341,2020-12-29,12,1,14,18,3,3,12,7,11,2,"[C443925, C601544, C158602, C722338, C792657, ...","[RECOVERED, RECOVERED, RECOVERED, RECOVERED, R...",382,382,0
342,2020-12-30,12,1,14,18,3,3,12,7,11,2,"[C676358, C795693, C415494, C878696, C437731, ...","[DIED, RECOVERED, RECOVERED, RECOVERED, RECOVE...",290,290,0
343,2020-12-31,12,1,14,18,3,3,12,7,11,2,"[C341025, C352062, C359056, C623135, C529531, ...","[RECOVERED, RECOVERED, RECOVERED, RECOVERED, R...",387,387,0


In [23]:
covid_cases_daily['CaseCnt_t-1'] = covid_cases_daily['CaseCnt'].shift()

In [24]:
covid_cases_daily.head()

,date,travel ban,travel ban lift,ease,postpone,emergency declaration,aid,protocol,testing,reopen,reporting change,CaseCode,RemovalType,CaseCnt,RemovalCnt,active_cases,CaseCnt_t-1
0,2020-01-23,1,0,0,0,0,0,0,0,0,0,[C387710],[RECOVERED],1,1,0,NaN
1,2020-01-24,1,0,0,0,0,0,0,0,0,0,[nan],[nan],0,0,0,1.0
2,2020-01-25,1,0,0,0,0,0,0,0,0,0,[nan],[nan],0,0,0,0.0
3,2020-01-26,1,0,0,0,0,0,0,0,0,0,[nan],[nan],0,0,0,0.0
4,2020-01-27,1,0,0,0,0,0,0,0,0,0,[nan],[nan],0,0,0,0.0


In [25]:
covid_cases_daily['growth_rate'] = round(((covid_cases_daily['CaseCnt']-\
                                         covid_cases_daily['CaseCnt_t-1'])/covid_cases_daily['CaseCnt_t-1'])*100,\
                                         2)
covid_cases_daily['growth_rate'] = np.where(covid_cases_daily['CaseCnt_t-1']==0,
                                            (covid_cases_daily['CaseCnt']-\
                                            covid_cases_daily['CaseCnt_t-1'])*100,
                                            covid_cases_daily['growth_rate'])

In [26]:
covid_cases_daily['growth_rate'] = covid_cases_daily['growth_rate'].fillna(0)

In [27]:
covid_cases_daily.tail(3)

,date,travel ban,travel ban lift,ease,postpone,emergency declaration,aid,protocol,testing,reopen,reporting change,CaseCode,RemovalType,CaseCnt,RemovalCnt,active_cases,CaseCnt_t-1,growth_rate
341,2020-12-29,12,1,14,18,3,3,12,7,11,2,"[C443925, C601544, C158602, C722338, C792657, ...","[RECOVERED, RECOVERED, RECOVERED, RECOVERED, R...",382,382,0,1057.0,-63.86
342,2020-12-30,12,1,14,18,3,3,12,7,11,2,"[C676358, C795693, C415494, C878696, C437731, ...","[DIED, RECOVERED, RECOVERED, RECOVERED, RECOVE...",290,290,0,382.0,-24.08
343,2020-12-31,12,1,14,18,3,3,12,7,11,2,"[C341025, C352062, C359056, C623135, C529531, ...","[RECOVERED, RECOVERED, RECOVERED, RECOVERED, R...",387,387,0,290.0,33.45


In [28]:
# lag values
for c_ in categories+['growth_rate']:
  covid_cases_daily[c_+'_lag'] = covid_cases_daily[c_].shift()

In [29]:
covid_cases_daily.tail(3)

,date,travel ban,travel ban lift,ease,postpone,emergency declaration,aid,protocol,testing,reopen,reporting change,CaseCode,RemovalType,CaseCnt,RemovalCnt,active_cases,CaseCnt_t-1,growth_rate,travel ban_lag,travel ban lift_lag,ease_lag,postpone_lag,emergency declaration_lag,aid_lag,protocol_lag,testing_lag,reopen_lag,reporting change_lag,growth_rate_lag
341,2020-12-29,12,1,14,18,3,3,12,7,11,2,"[C443925, C601544, C158602, C722338, C792657, ...","[RECOVERED, RECOVERED, RECOVERED, RECOVERED, R...",382,382,0,1057.0,-63.86,11.0,1.0,14.0,18.0,3.0,3.0,12.0,7.0,11.0,2.0,69.39
342,2020-12-30,12,1,14,18,3,3,12,7,11,2,"[C676358, C795693, C415494, C878696, C437731, ...","[DIED, RECOVERED, RECOVERED, RECOVERED, RECOVE...",290,290,0,382.0,-24.08,12.0,1.0,14.0,18.0,3.0,3.0,12.0,7.0,11.0,2.0,-63.86
343,2020-12-31,12,1,14,18,3,3,12,7,11,2,"[C341025, C352062, C359056, C623135, C529531, ...","[RECOVERED, RECOVERED, RECOVERED, RECOVERED, R...",387,387,0,290.0,33.45,12.0,1.0,14.0,18.0,3.0,3.0,12.0,7.0,11.0,2.0,-24.08


In [30]:
covid_cases_daily['CaseCnt_log'] = covid_cases_daily['CaseCnt'].apply(lambda x: np.log(x) if x>0 else 0)
covid_cases_daily['CaseCnt_t-1_log'] = covid_cases_daily['CaseCnt_t-1'].apply(lambda x: np.log(x) if x>0 else 0)

In [31]:
covid_cases_daily['growth_rate_SIR'] = covid_cases_daily['CaseCnt_log']-covid_cases_daily['CaseCnt_t-1_log']

In [32]:
covid_cases_daily.tail(3)

,date,travel ban,travel ban lift,ease,postpone,emergency declaration,aid,protocol,testing,reopen,reporting change,CaseCode,RemovalType,CaseCnt,RemovalCnt,active_cases,CaseCnt_t-1,growth_rate,travel ban_lag,travel ban lift_lag,ease_lag,postpone_lag,emergency declaration_lag,aid_lag,protocol_lag,testing_lag,reopen_lag,reporting change_lag,growth_rate_lag,CaseCnt_log,CaseCnt_t-1_log,growth_rate_SIR
341,2020-12-29,12,1,14,18,3,3,12,7,11,2,"[C443925, C601544, C158602, C722338, C792657, ...","[RECOVERED, RECOVERED, RECOVERED, RECOVERED, R...",382,382,0,1057.0,-63.86,11.0,1.0,14.0,18.0,3.0,3.0,12.0,7.0,11.0,2.0,69.39,5.945421,6.963190,-1.017769
342,2020-12-30,12,1,14,18,3,3,12,7,11,2,"[C676358, C795693, C415494, C878696, C437731, ...","[DIED, RECOVERED, RECOVERED, RECOVERED, RECOVE...",290,290,0,382.0,-24.08,12.0,1.0,14.0,18.0,3.0,3.0,12.0,7.0,11.0,2.0,-63.86,5.669881,5.945421,-0.275540
343,2020-12-31,12,1,14,18,3,3,12,7,11,2,"[C341025, C352062, C359056, C623135, C529531, ...","[RECOVERED, RECOVERED, RECOVERED, RECOVERED, R...",387,387,0,290.0,33.45,12.0,1.0,14.0,18.0,3.0,3.0,12.0,7.0,11.0,2.0,-24.08,5.958425,5.669881,0.288544


In [33]:
covid_cases_daily.to_csv('/content/drive/MyDrive/MA in Economics/Econ 232 Econometrics/term_paper/covid_cases_daily_2.csv')

## EDA

In [34]:
print(f'Total number of days included in the data: {covid_cases_daily.shape[0]}')
print('Descriptive Statistics for each variable:')

summary_stats = []
for col in ['CaseCnt', 'growth_rate']+categories:
  sstats = covid_cases_daily[col].describe()
  summary_stats.append(sstats.to_frame().transpose())

summary_stats = pd.concat(summary_stats)
display(summary_stats)

Total number of days included in the data: 344
Descriptive Statistics for each variable:


,count,mean,std,min,25%,50%,75%,max
CaseCnt,344.0,1376.898256,1236.355110,0.0,295.250,1105.0,2154.500,5689.0
growth_rate,344.0,13.611395,103.733302,-100.0,-17.325,0.0,14.045,1700.0
travel ban,344.0,8.848837,2.014625,1.0,9.000,10.0,10.000,12.0
travel ban lift,344.0,0.936047,0.245026,0.0,1.000,1.0,1.000,1.0
ease,344.0,3.148256,3.880669,0.0,1.000,1.0,3.000,14.0
postpone,344.0,9.875000,5.187905,0.0,6.000,12.0,14.000,18.0
emergency declaration,344.0,2.017442,0.947446,0.0,2.000,2.0,3.000,3.0
aid,344.0,2.427326,1.127684,0.0,3.000,3.0,3.000,3.0
protocol,344.0,4.142442,4.283680,0.0,1.000,2.0,9.000,12.0
testing,344.0,4.840116,2.791181,0.0,3.000,7.0,7.000,7.0
